In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# colab pro 더 빠른 GPU 사용
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Not connected to a GPU


In [3]:
# colab pro 추가 메모리
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 13.6 gigabytes of available RAM

Not using a high-RAM runtime


In [4]:
# nltk에서 stopwords 다운로드
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [5]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import os
import re
import numpy as np
import pandas as pd
import time
#import pyLDAvis
#import pyLDAvis.gensim_models
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

from nltk.tokenize import RegexpTokenizer

from sklearn.feature_extraction.text import TfidfVectorizer
from wordcloud import WordCloud
from matplotlib import pyplot as plt

import gensim
import string
from gensim import corpora
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

import multiprocessing
num_cores = multiprocessing.cpu_count()

# Change this to where your datasets are located:
os.chdir("/content/drive/MyDrive/Colab Notebooks/Aiffel_Hackathon/financial_documents/training_gold_standards") # 현재 디렉토리 설정. 잘못 건드리면 open에서 문제생길 수 있음.

## 확인 필요. 문제 없으면 지울 수 있음. ##
# Check versions to ensure that there are no compatability issues:
python_version = !python --version
print("Python Version: ", python_version)
print("Current Directory: ", os.getcwd())
print("Numpy version: ", np.__version__)
print("Pandas version: ", pd.__version__)

Python Version:  ['Python 3.7.12']
Current Directory:  /content/drive/.shortcut-targets-by-id/1w4WyhAQ16CIFKNbtXntCgKMZoYxIpzws/Aiffel_Hackathon/financial_documents/training_gold_standards
Numpy version:  1.19.5
Pandas version:  1.1.5


파일 하나로 테스트

In [89]:
# training_section 3번째 파일로 preprocessing test 진행
test_file_path1 = '/content/drive/MyDrive/Colab Notebooks/Aiffel_Hackathon/financial_documents/training_sections/64_868733_2.txt'
temp_str = ''
temp_list = []
with open(test_file_path1, "r", encoding='utf-8') as f:
  for line in f:
    # 끝에는 띄어쓰기로 치환. 두번 나와도 나중에 정규식으로 제거하면 됨.
    line = line.replace('\n', ' ')
    temp_str += line
temp_list.append(temp_str)

In [90]:
temp_list

[' VPhase plc   Group Financial Statements for the year ended 31 December 2008 05 Chief Executive’s Review I am delighted to report that the Group has successfully delivered  its key objective of positioning its first product VX1 to be ready for  the next stage of commercialisation in 2009 and to place VX1 in  an Ofgem Demonstration Action programme. This programme  has been fully funded by SSE to a value of up to £240,000, and it  is anticipated that these trials of VX1 will demonstrate significant  carbon dioxide savings when used in UK domestic properties.  Additionally, the Group is starting a programme of commercial  partnerships to introduce VPhase initially to European and then to  other geographic markets. We have already entered into a  memorandum of understanding with British Gas to consider  routes to market for VPhase products in the UK and also into  North America through Direct Energy, one of America’s leading  integrated energy companies and another Centrica-owned  busin

리스트에 넣어서 iteration 돌리기

In [105]:
origin_path = '/content/drive/MyDrive/Colab Notebooks/Aiffel_Hackathon/financial_documents/preprocessing_test/'
test_file_temp_list = os.listdir('/content/drive/MyDrive/Colab Notebooks/Aiffel_Hackathon/financial_documents/preprocessing_test/')
test_file_temp_list

test_file_list = []
for i in test_file_temp_list:
  a = origin_path + i
  test_file_list.append(a)  

['17_896314_2.txt',
 '64_868733_2.txt',
 '66_868765_2.txt',
 '19_896332_2.txt',
 '65_868749_2.txt']

In [109]:
test_file_list

['/content/drive/MyDrive/Colab Notebooks/Aiffel_Hackathon/financial_documents/preprocessing_test/17_896314_2.txt',
 '/content/drive/MyDrive/Colab Notebooks/Aiffel_Hackathon/financial_documents/preprocessing_test/64_868733_2.txt',
 '/content/drive/MyDrive/Colab Notebooks/Aiffel_Hackathon/financial_documents/preprocessing_test/66_868765_2.txt',
 '/content/drive/MyDrive/Colab Notebooks/Aiffel_Hackathon/financial_documents/preprocessing_test/19_896332_2.txt',
 '/content/drive/MyDrive/Colab Notebooks/Aiffel_Hackathon/financial_documents/preprocessing_test/65_868749_2.txt']

In [125]:
sentence_list = []

for i in test_file_list:
  temp_str = ''
  with open(i, "r", encoding='utf-8') as f:
    for line in f:
      # 끝에는 띄어쓰기로 치환. 두번 나와도 나중에 정규식으로 제거하면 됨.
      line = line.replace('\n', ' ')
      temp_str += line
    sentence_list.append(temp_str)

In [126]:
sentence_list

['  06 Z GROUP plc  ANNUAL REPORT AND ACCOUNTS 2006 JOINT CHIEF EXECUTIVES’ STATEMENT JAMIE TRUE Joint Chief Executive JACK BEKHOR Joint Chief Executive “ This year has seen us achieve not only admission to AIM  but also a 78% increase in our proﬁ ts, with a pre-tax proﬁ t  of £1.1 million.” OVERVIEW This is our ﬁ rst Annual Report since joining AIM and we are  pleased to announce that we have achieved further  improvement in our proﬁ ts this year, our third consecutive  year of growth, with a pre-tax proﬁ t of £1.1 million. This  success has been the result of consolidating our core  business, developing new delivery channels and expanding  into European and other international markets. We have  also made signiﬁ cant investment in our product portfolio,  both by licensing new technologies to commercialise  and by developing in-house proprietary technologies to  address the needs of mass consumer markets. AIMS AND ACHIEVEMENTS Our primary aim this year was to increase our turnover  and

\s\s(엔터 두번) 들어가는 요소들이 있음

In [130]:
sentence_list[0]

'  06 Z GROUP plc  ANNUAL REPORT AND ACCOUNTS 2006 JOINT CHIEF EXECUTIVES’ STATEMENT JAMIE TRUE Joint Chief Executive JACK BEKHOR Joint Chief Executive “ This year has seen us achieve not only admission to AIM  but also a 78% increase in our proﬁ ts, with a pre-tax proﬁ t  of £1.1 million.” OVERVIEW This is our ﬁ rst Annual Report since joining AIM and we are  pleased to announce that we have achieved further  improvement in our proﬁ ts this year, our third consecutive  year of growth, with a pre-tax proﬁ t of £1.1 million. This  success has been the result of consolidating our core  business, developing new delivery channels and expanding  into European and other international markets. We have  also made signiﬁ cant investment in our product portfolio,  both by licensing new technologies to commercialise  and by developing in-house proprietary technologies to  address the needs of mass consumer markets. AIMS AND ACHIEVEMENTS Our primary aim this year was to increase our turnover  and 

In [150]:
re.sub('\s\s', ' ' ,sentence_list[0])

' 06 Z GROUP plc ANNUAL REPORT AND ACCOUNTS 2006 JOINT CHIEF EXECUTIVES’ STATEMENT JAMIE TRUE Joint Chief Executive JACK BEKHOR Joint Chief Executive “ This year has seen us achieve not only admission to AIM but also a 78% increase in our proﬁ ts, with a pre-tax proﬁ t of £1.1 million.” OVERVIEW This is our ﬁ rst Annual Report since joining AIM and we are pleased to announce that we have achieved further improvement in our proﬁ ts this year, our third consecutive year of growth, with a pre-tax proﬁ t of £1.1 million. This success has been the result of consolidating our core business, developing new delivery channels and expanding into European and other international markets. We have also made signiﬁ cant investment in our product portfolio, both by licensing new technologies to commercialise and by developing in-house proprietary technologies to address the needs of mass consumer markets. AIMS AND ACHIEVEMENTS Our primary aim this year was to increase our turnover and profits whilst 

정규표현식으로 제거

In [158]:
sentence_list2 = []
for i in sentence_list:
  a = re.sub('\s\s', ' ', i)
  sentence_list2.append(a)
sentence_list2

[' 06 Z GROUP plc ANNUAL REPORT AND ACCOUNTS 2006 JOINT CHIEF EXECUTIVES’ STATEMENT JAMIE TRUE Joint Chief Executive JACK BEKHOR Joint Chief Executive “ This year has seen us achieve not only admission to AIM but also a 78% increase in our proﬁ ts, with a pre-tax proﬁ t of £1.1 million.” OVERVIEW This is our ﬁ rst Annual Report since joining AIM and we are pleased to announce that we have achieved further improvement in our proﬁ ts this year, our third consecutive year of growth, with a pre-tax proﬁ t of £1.1 million. This success has been the result of consolidating our core business, developing new delivery channels and expanding into European and other international markets. We have also made signiﬁ cant investment in our product portfolio, both by licensing new technologies to commercialise and by developing in-house proprietary technologies to address the needs of mass consumer markets. AIMS AND ACHIEVEMENTS Our primary aim this year was to increase our turnover and profits whilst